In [ ]:
# Turning 30 separate CSV files into one Dataframe containing all pitches from 2023 season
import pandas as pd
import os

csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

pitch_df = pd.DataFrame()

for file in csv_files:
  df = pd.read_csv(file)
  pitch_df = pd.concat([pitch_df,df])

pitch_df = pitch_df.reset_index(drop = True)

pitch_df.to_csv('pitch_df.csv', index=False)
from google.colab import files
files.download('pitch_df.csv')

pitch_df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2023-10-01,82.9,-1.56,6.22,"Giolito, Lucas",595879,608337,field_out,hit_into_play,...,2,2,5,5,2,Standard,Standard,52.0,-0.004,-0.121
1,SL,2023-10-01,83.7,-1.53,6.17,"Giolito, Lucas",595879,608337,NaN,swinging_strike,...,2,2,5,5,2,Standard,Standard,25.0,0.000,-0.023
2,CH,2023-10-01,82.0,-1.26,6.24,"Giolito, Lucas",595879,608337,NaN,ball,...,2,2,5,5,2,Standard,Standard,226.0,0.000,0.022
3,CH,2023-10-01,82.9,-1.58,6.13,"Giolito, Lucas",595879,608337,NaN,ball,...,2,2,5,5,2,Standard,Standard,226.0,0.000,0.016
4,CU,2023-10-01,77.3,-1.41,6.38,"Giolito, Lucas",668731,608337,field_out,hit_into_play,...,2,2,5,5,2,Standard,Standard,21.0,-0.007,-0.148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713909,SI,2023-05-19,94.1,2.49,5.61,"Matz, Steven",605141,571927,NaN,ball,...,0,0,0,0,0,Standard,Standard,117.0,0.000,0.041
713910,FF,2023-05-19,91.1,-2.04,5.86,"Stratton, Chris",681546,608717,NaN,swinging_strike,...,0,1,0,1,0,Infield shade,Standard,214.0,0.000,-0.063
713911,SL,2023-05-19,81.2,2.93,6.02,"Vasquez, Andrew",673548,607755,single,hit_into_play,...,1,8,1,8,1,Standard,Standard,307.0,-0.003,0.733
713912,FC,2023-05-19,91.9,-2.09,5.64,"Holderman, Colin",672695,670059,NaN,swinging_strike,...,13,3,13,3,13,Infield shade,Standard,207.0,0.000,-0.067


In [ ]:
# Assuming 'pitch_df' is your DataFrame containing every pitch in 2023

# Modify the 'pfx_x' column to contain absolute values
pitch_df['pfx_x'] = pitch_df['pfx_x'].abs()

# Now 'pfx_x' contains absolute values
print(pitch_df.head())

  pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0         SL  2023-10-01           82.9          -1.56           6.22   
1         SL  2023-10-01           83.7          -1.53           6.17   
2         CH  2023-10-01           82.0          -1.26           6.24   
3         CH  2023-10-01           82.9          -1.58           6.13   
4         CU  2023-10-01           77.3          -1.41           6.38   

      player_name  batter  pitcher     events      description  ...  \
0  Giolito, Lucas  595879   608337  field_out    hit_into_play  ...   
1  Giolito, Lucas  595879   608337        NaN  swinging_strike  ...   
2  Giolito, Lucas  595879   608337        NaN             ball  ...   
3  Giolito, Lucas  595879   608337        NaN             ball  ...   
4  Giolito, Lucas  668731   608337  field_out    hit_into_play  ...   

   fld_score  post_away_score  post_home_score  post_bat_score  \
0          2                2                5               5   
1 

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('pitches_df.db')
table_name = 'pitches_table'
pitch_df.to_sql(table_name, conn, if_exists='replace', index=False)

desired_descriptions = ['foul', 'swinging_strike', 'hit_into_play', 'foul_tip', 'swinging_strike_blocked']

query_RHP_FF_RHB = f"""
    SELECT description, release_speed, pfx_x, pfx_z, zone
    FROM pitches_table
    WHERE pitch_type = 'FF' AND p_throws = 'R' AND stand = 'R' AND description IN {tuple(desired_descriptions)}
"""

RHP_FF_RHB = pd.read_sql_query(query_RHP_FF_RHB, conn)

print(RHP_FF_RHB)


In [ ]:
from scipy import stats

# Calculate z-scores for release speed
RHP_FF_RHB['z_score'] = stats.zscore(RHP_FF_RHB['release_speed'])

# Set a threshold for excluding outliers
threshold = -3

# Filter out rows with z-scores below the threshold
filtered_RHP_FF_RHB = RHP_FF_RHB[RHP_FF_RHB['z_score'] > threshold]

# Drop rows with incomplete data
filtered_RHP_FF_RHB = filtered_RHP_FF_RHB.dropna()

# Drop the 'z_score' column from the final DataFrame
filtered_RHP_FF_RHB = filtered_RHP_FF_RHB.drop(columns=['z_score'])

print(filtered_RHP_FF_RHB)


           description  release_speed  pfx_x  pfx_z  zone
0                 foul           92.1   0.55   1.09  11.0
1      swinging_strike           92.2   0.67   1.18   6.0
2        hit_into_play           94.2   0.50   1.20   3.0
3      swinging_strike           93.9   0.28   1.20  12.0
4      swinging_strike           94.1   0.38   1.44   2.0
...                ...            ...    ...    ...   ...
39072    hit_into_play           94.0   0.68   1.74   5.0
39073  swinging_strike           92.3   0.25   1.43   3.0
39074  swinging_strike           94.0   1.03   1.23   2.0
39075    hit_into_play           93.5   0.93   1.13   2.0
39076             foul           93.3   0.47   1.56  14.0

[38968 rows x 5 columns]


In [ ]:
#Finding 20,40,60,80,100 percentiles of HB, VB, and RS
import numpy as np

# Calculate percentile values
release_speed_percentiles = np.percentile(filtered_RHP_FF_RHB['release_speed'], [0, 20, 40, 60, 80, 100])
pfx_x_percentiles = np.percentile(filtered_RHP_FF_RHB['pfx_x'], [0, 20, 40, 60, 80, 100])
pfx_z_percentiles = np.percentile(filtered_RHP_FF_RHB['pfx_z'], [0, 20, 40, 60, 80, 100])

# Calculate average values
release_speed_average = np.mean(filtered_RHP_FF_RHB['release_speed'])
pfx_x_average = np.mean(filtered_RHP_FF_RHB['pfx_x'])
pfx_z_average = np.mean(filtered_RHP_FF_RHB['pfx_z'])

# Print percentile and average values
print('Release Speed Percentiles:')
print(release_speed_percentiles)
print('Release Speed Average:')
print(release_speed_average)
print('PFX_X Percentiles:')
print(pfx_x_percentiles)
print('PFX_X Average:')
print(pfx_x_average)
print('PFX_Z_Percentiles:')
print(pfx_z_percentiles)
print('pfx_z Average:')
print(pfx_z_average)

Release Speed Percentiles:
[ 87.9  92.9  94.1  95.3  96.7 104.8]
Release Speed Average:
94.77553890371587
PFX_X Percentiles:
[0.   0.34 0.54 0.69 0.86 1.64]
PFX_X Average:
0.6074294292753027
PFX_Z_Percentiles:
[0.21 1.16 1.3  1.41 1.53 2.14]
pfx_z Average:
1.3402117121740917


In [ ]:
import numpy as np
import pandas as pd

# Calculate release speed percentiles
release_speed_percentiles = np.percentile(filtered_RHP_FF_RHB['release_speed'], [0, 20, 40, 60, 80, 100])

# Use Pandas' cut function to create categories
speed_categories = pd.cut(filtered_RHP_FF_RHB['release_speed'], bins=release_speed_percentiles, labels=False, precision=0)

# Define labels for the categories
category_labels = ['Poor Velocity', 'Below Average Velocity', 'Average Velocity', 'Above Average Velocity', 'Elite Velocity']

# Split dataframe based on categories
dataframes_by_speed_category = [filtered_RHP_FF_RHB[speed_categories == i] for i in range(len(category_labels))]

# Assign each dataframe to a separate variable
df_poor_velocity = dataframes_by_speed_category[0]
df_below_average_velocity = dataframes_by_speed_category[1]
df_average_velocity = dataframes_by_speed_category[2]
df_above_average_velocity = dataframes_by_speed_category[3]
df_elite_velocity = dataframes_by_speed_category[4]

# Ensure that each category has a corresponding dataframe
list_of_dataframes = [df_poor_velocity, df_below_average_velocity, df_average_velocity, df_above_average_velocity, df_elite_velocity]


In [ ]:

PFX_Z_Percentiles = [0.05, 1.16, 1.3, 1.41, 1.53, 2.29]

for df in list_of_dataframes:
    # Add VB_Group feature based on PFX_Z_Percentiles
    df['VM_Group'] = pd.cut(df['pfx_z'], bins=PFX_Z_Percentiles, labels=['Poor', 'Below Average', 'Average', 'Above Average', 'Elite'], include_lowest=True)

# Now, each dataframe in list_of_dataframes has a new 'VB_Group' feature.
df_poor_velocity

<ipython-input-40-64ffad26ab2d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VM_Group'] = pd.cut(df['pfx_z'], bins=PFX_Z_Percentiles, labels=['Poor', 'Below Average', 'Average', 'Above Average', 'Elite'], include_lowest=True)
<ipython-input-40-64ffad26ab2d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VM_Group'] = pd.cut(df['pfx_z'], bins=PFX_Z_Percentiles, labels=['Poor', 'Below Average', 'Average', 'Above Average', 'Elite'], include_lowest=True)
<ipython-input-40-64ffad26ab2d>:5: SettingWi

,description,release_speed,pfx_x,pfx_z,zone,VM_Group
0,foul,92.1,0.55,1.09,11.0,Poor
1,swinging_strike,92.2,0.67,1.18,6.0,Below Average
12,swinging_strike,91.2,0.78,0.98,5.0,Poor
13,foul,91.8,0.06,1.22,6.0,Below Average
16,foul,92.6,0.51,1.56,8.0,Elite
...,...,...,...,...,...,...
39051,hit_into_play,91.6,0.65,1.28,2.0,Below Average
39053,swinging_strike,92.2,0.22,1.50,5.0,Above Average
39056,swinging_strike,92.3,0.69,1.27,1.0,Below Average
39062,hit_into_play,92.2,0.55,1.11,2.0,Poor
